# End-to-End Analysis and Visualization of E3SM Data using nco and xCDAT

E3SM Tutorial Workshop 2024

05/07/2024

Authors: [Tom Vo](https://github.com/tomvothecoder) and [Stephen Po-Chedley](https://github.com/pochedls)


## Overview

This exercise notebook will walkthrough using regridding E3SM data to a
rectilinear grid using `ncremap`, then performing analysis and visualization using xCDAT.

### Sections

1. Prerequisite: Set up the E3SM Unified Environment v1.10.0 Python Kernel
2. Setup Code
3. Use NCO to regrid E3SM data to a rectilinear grid
4. I/O
5. Regridding
6. Spatial Averaging
7. Temporal Computations
8. General Dataset Utilities


## Prerequisite: Set up the E3SM Unified Environment Kernel

_Skip this section if you've already done it_

1. Run the cell below to add the E3SM Unified Environment v1.10.0 kernel to Jupyter Hub 

In [ ]:
%%bash

# Source https://docs.nersc.gov/services/jupyter/how-to-guides/#how-to-use-a-conda-environment-as-a-python-kernel
# Activate the E3SM Unified Environment
source /global/common/software/e3sm/anaconda_envs/load_latest_e3sm_unified_pm-cpu.sh

# Add the E3SM Unified Environment kernel to Jupyter Hub
python -m ipykernel install \
--user --name e3sm_unified_1.10.0 --display-name e3sm_unified_1.10.0


3. Refresh this page.
4. Select the kernel for this notebook by clicking the current kernel in the top-right
   (where it says NERSC Python in the screenshot).

   <img src="kernel-instructions-1.png" width=500px/>

5. Select `e3sm_unified.1.10.0` from the list of environments.

   <img src="kernel-instructions-2.png" width=500px/>


## Setup Code


In [ ]:
import glob

import numpy as np
from xarray.coding.calendar_ops import _datetime_to_decimal_year as dt2decimal
import xcdat as xc
import cartopy.crs as ccrs
import matplotlib.pyplot as plt


## Use NCO to regrid E3SM data to a rectilinear grid


### Now call ncremap to regrid the file to a 0.5 x 0.5 degree grid

Typically a user would call this command directly from the shell or write a batch script to run ncremap. Here we use the bash decorator in Jupyter to run `ncremap` on a directory of files (using a wildcard to filter for `.h0` files, which include monthly output we’d like to analyze). We will then move the remapped files to a `remapped/` directory.


In [ ]:
%%bash
# create output directory
mkdir -p remapped
# source e3sm-unified environment
source /global/common/software/e3sm/anaconda_envs/load_latest_e3sm_unified_pm-cpu.sh
# do regridding
# format: ncremap -m REMAPFILE.nc -t 1 -v VAR_OF_INTEREST /PATH/TO/DATA/*nc
# Subsetting files for "h0", which is the monthly history field.
ncremap -m /global/cfs/cdirs/e3sm/diagnostics/maps/map_ne30pg2_to_cmip6_180x360_aave.20200201.nc -t 1 -v TREFHT /global/cfs/cdirs/e3sm/www/Tutorials/2024/simulations/extendedOutput.v3.LR.historical_0101/archive/atm/hist/extendedOutput.v3.LR.historical_0101.eam.h0.*nc >/dev/null 2>&1
# move output to remapped directory
mv extendedOutput.v3.LR.historical_0101.eam.h0.*nc remapped/

## I/O


### Now let's load in the regridded data and use xcdat to perform additional calculations on the 0.5 x 0.5 degree grid


#### 💻 Your turn:

Use `xc.open_mfdataset()` to open all the remapped netcdf files in a single `xr.Dataset` object. With `xcdat`, you can specify the directory `remapped` and xcdat will read in all netcdf files as one `xr.dataset` object. You could also use a wildcard with xarray (`ds = xr.open_mfdataset('remapped/*.nc’)`). `open_mfdataset` is essentially the same operation in both Xarray and xCDAT, but `xcdat` will add missing bounds and handles some additional time axes. 

We will be analyzing a few years of temperature (`TREFHT`) data from E3SM v3.

- Documentation: https://xcdat.readthedocs.io/en/stable/generated/xcdat.open_mfdataset.html 


In [ ]:
# Your code here. When ready, click on the three dots below for the solution.
ds = 

In [ ]:
ds = xc.open_mfdataset("remapped")
ds

...but checkout the time coordinate:


In [ ]:
ds.time.values[0:3]

The monthly time coordinates begin in 2/2000, even though our first file is for 1/2000. This is because E3SM saves out monthly history at midnight at the end of the month. xCDAT can handle this by centering the time coordinates between the monthly time bounds (using `center_times=True`):


#### 💻 Your turn:

Use `xc.open_mfdataset()` again, but make sure to center the time coordinates.

- Documentation: https://xcdat.readthedocs.io/en/latest/generated/xcdat.open_mfdataset.html#xcdat.open_mfdataset
- Hint: Use the `center_times` arg


In [ ]:
# Your code here. When ready, click on the three dots below for the solution.
ds = 

In [ ]:
ds = xc.open_mfdataset("remapped", center_times=True)
ds

## Regridding with xCDAT

We often want to regrid a dataset to a new grid to facilitate data analysis or comparisons with other datasets. The current dataset is at 0.5 x 0.5<sup>o</sup> resolution, so we'll start be remapping it to a 4 x 4<sup>o</sup> grid.


### First, we specify the target grid


In [ ]:
# create target axes
nlat = xc.create_axis(
    "lat", np.arange(-88, 90, 4), attrs={"units": "degrees_north", "axis": "Y"}
)
nlon = xc.create_axis(
    "lon", np.arange(2, 360, 4), attrs={"units": "degrees_east", "axis": "X"}
)

#### 💻 Your turn:

Create the target grid using the target axes and bounds.

- Documentation: https://xcdat.readthedocs.io/en/latest/generated/xcdat.create_grid.html#xcdat.create_grid


In [ ]:
# Your code here. When ready, click on the three dots below for the solution.
ngrid = xc.create_grid()

In [ ]:
ngrid = xc.create_grid(x=nlon, y=nlat)

### Call the xESMF regridder

Here we're using bilinear regridding, but other methods may be appropriate (e.g., you may want to use "conservative_normed" for fields that should be conserved globally).


#### 💻 Your turn:

Regrid "TREFHT" with the `ngrid` created above using `xesmf` and `bilinear`.

- API Documentation: https://xcdat.readthedocs.io/en/stable/generated/xarray.Dataset.regridder.horizontal.html


In [ ]:
# Starter code here. When ready, click on the three dots below for the solution.
ds_xesmf = ds.regridder.horizontal()

In [ ]:
ds_xesmf = ds.regridder.horizontal("TREFHT", ngrid, tool="xesmf", method="bilinear")

### Compare the results (for the first timestep)

Now we just plot the results for comparison.


In [ ]:
map_proj = ccrs.Robinson()

# plot original data (first time step)
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1, projection=map_proj)
p = ds.TREFHT[0].plot(
    transform=ccrs.PlateCarree(),  # the data's projection
    subplot_kws={"projection": map_proj},
    cbar_kwargs={"orientation": "horizontal"},
    cmap=plt.cm.RdBu_r,
)
ax = plt.gca()
ax.coastlines()
plt.title("Original")

# plot the remapped data (first time step)
plt.subplot(1, 2, 2, projection=map_proj)
p = ds_xesmf.TREFHT[0].plot(
    transform=ccrs.PlateCarree(),  # the data's projection
    subplot_kws={"projection": map_proj},
    cbar_kwargs={"orientation": "horizontal"},
    cmap=plt.cm.RdBu_r,
)
ax = plt.gca()
ax.coastlines()
plt.title("xESMF 4$^{\circ}$ x 4$^{\circ}$")

### Vertical Regridding

xcdat can also regrid in the vertical. Here we'll grab some 3D temperature data and regrid it in the vertical. First, we need to remap some 3-dimensional data to a rectilinear grid (like we did for the surface air temperature data, `TREFHT`).


In [ ]:
%%bash
# source e3sm-unified environment
source /global/common/software/e3sm/anaconda_envs/load_latest_e3sm_unified_pm-cpu.sh
# remap (we are only remapping one file and specifying the output location)
ncremap -m /global/cfs/cdirs/e3sm/diagnostics/maps/map_ne30pg2_to_cmip6_180x360_aave.20200201.nc -t 1 -v T /global/cfs/cdirs/e3sm/www/Tutorials/2024/simulations/extendedOutput.v3.LR.historical_0101/archive/atm/hist/extendedOutput.v3.LR.historical_0101.eam.h0.2000-01.nc T_extendedOutput.v3.LR.historical_0101.eam.h0.2000-01.nc >/dev/null 2>&1

Now let's load the data:


In [ ]:
# specify file we just regridded
fn = "T_extendedOutput.v3.LR.historical_0101.eam.h0.2000-01.nc"

# load regridded data
ds3d = xc.open_dataset(fn)

Next, we will do the vertical remapping...


In [ ]:
# first construct the 3D pressure field
pressure = ds3d["hyam"] * 1000.0 + ds3d["hybm"] * ds3d["PS"]

# next, construct the target pressure axis
target_plevs = [
    100000,
    92500,
    85000,
    75000,
    70000,
    60000,
    50000,
    40000,
    30000,
    25000,
    20000,
    15000,
    10000,
    7000,
    5000,
    3000,
    1000,
    500,
    300,
    100,
]
nplev = xc.create_grid(z=xc.create_axis("lev", target_plevs))

#### 💻 Your turn:

Regrid the `"T"` variable using `nplev` as the output grid, `"log"` method, and `pressure` as the target data.

- Example Documentation: https://xcdat.readthedocs.io/en/stable/examples/regridding-vertical.html#4:-Remap-cloud-fraction-from-model-hybrid-coordinate-to-pressure-levels


In [ ]:
# Starter code here. When ready, click on the three dots below for the solution.
dsvr = ds3d.regridder.vertical(...)

In [ ]:
dsvr = ds3d.regridder.vertical("T", nplev, method="log", target_data=pressure)

Finally, we plot the result:


In [ ]:
# plot result
dsvr_zonal = dsvr.spatial.average("T", axis=["X"]).squeeze()
dsvr_zonal.T.plot(cmap=plt.cm.RdBu_r)
plt.gca().invert_yaxis()

## Spatial Averaging with xCDAT

Area-weighted spatial averaging is a common technique to reduce dimensionality in geospatial datasets. xCDAT can perform this calculation over full domains or regions of interest.


#### 💻 Your turn:

Calculate the spatial average of "TREFHT" and store the results in a Python variable.

- API Documentation: https://xcdat.readthedocs.io/en/stable/generated/xarray.Dataset.spatial.average.html


In [ ]:
# Your code here. When ready, click on the three dots below for the solution.
ds_global = ds.spatial.average()

In [ ]:
ds_global = ds.spatial.average("TREFHT")

#### Now let's plot the results.

Note that the spatial averager returns a dataset object so we still need to specify "TREFHT" to plot the dataarray.


In [ ]:
dtime = dt2decimal(ds_global.time)  # decimal time
plt.plot(dtime, ds_global["TREFHT"].values)
plt.xlabel("Year")
plt.ylabel("Global Mean Temperature [K]")

Above, we did not specify any constraints. So xCDAT calculated the domain (global) average. Users can also specify their own bounds.


#### 💻 Your turn:

Calculate the the average surface temperature (`"TREFHT"`) in the Niño 3.4 region.

- API Documentation: https://xcdat.readthedocs.io/en/stable/generated/xarray.Dataset.spatial.average.html
- Hint: Pass latitude bounds of (-5, 5) and longitude bounds of (190, 240) and keep the weights.


In [ ]:
# Starter code here. When ready, click on the three dots below for the solution.
ds_nino34 = ds_xesmf.spatial.average()

In [ ]:
ds_nino34 = ds_xesmf.spatial.average(
    "TREFHT", lat_bounds=(-5, 5), lon_bounds=(190, 240), keep_weights=True
).load()

In this case, we specified `keep_weights=True`. The weights provide full spatial weighting for grid cells entirely within the Niño 3.4 region. If a grid cell is partially in the Niño 3.4 region, it received partial weight (note we use the 4 x 4 degree grid in this example to show the partial weights and to speed up plotting). Note that you can also supply your own weights (but you can't automatically subset with `lat_bounds` and `lon_bounds` if you supply your own weights).


In [ ]:
# show the nino 3.4 time series
plt.figure(figsize=(10, 2))
plt.subplot(1, 2, 1)
plt.plot(dtime, ds_nino34["TREFHT"].values)
plt.xlabel("Year")
plt.ylabel("Surface Temperature [K]")
plt.title("Niño 3.4 time series")

# show the weights
map_proj = ccrs.PlateCarree(central_longitude=180)
ax = plt.subplot(1, 2, 2, projection=map_proj)
plt.pcolor(
    ds_nino34.lon,
    ds_nino34.lat,
    ds_nino34.lat_lon_wts.T,
    transform=ccrs.PlateCarree(),
    cmap=plt.cm.GnBu,
)
ax.set_extent([120, 300, -30, 30], crs=ccrs.PlateCarree())
ax.coastlines()
plt.colorbar(orientation="horizontal")
plt.title("Nino 3.4 Weights")

## Temporal Computations with xCDAT

In the examples below, we will performing temporal computations on the `xarray.Dataset` object using xCDAT.


### Annual cycle

In the global mean time series above, there are large seasonal swings in global temperature. Here we compute the seasonal mean climatology.


#### 💻 Your turn:

Calculate the seasonal mean climatology for the `"TREFHT"` variable.

- API Documentation: https://xcdat.readthedocs.io/en/stable/generated/xarray.Dataset.temporal.climatology.html


In [ ]:
# Starter code here. When ready, click on the three dots below for the solution.
ds_clim = ds.temporal.climatology()

In [ ]:
# compute the climatology
ds_clim = ds.temporal.climatology("TREFHT", freq="season")

### Now we plot the season means


In [ ]:
map_proj = ccrs.Robinson()
titles = ["DJF", "MAM", "JJA", "SON"]
plt.figure(figsize=(12, 10))
for i in range(4):
    plt.subplot(2, 2, i + 1, projection=map_proj)
    p = ds_clim.TREFHT[i].plot(
        transform=ccrs.PlateCarree(),
        subplot_kws={"projection": map_proj},
        cbar_kwargs={"orientation": "horizontal"},
        cmap=plt.cm.RdBu_r,
        vmin=220,
        vmax=310,
    )
    ax = plt.gca()
    ax.coastlines()
    plt.title(titles[i])

### Departures


#### 💻 Your turn:

It can also be useful to show the departures from the climatological average.

Calculate the seasonal mean climatology for the `"TREFHT"` variable. In this case, `xcdat` will operate on the global mean time series we calculated above. Note that you can set the climatological reference period (e.g., with `reference_period=("1950-01-01", "1999-12-31")` for historical era departures).

- API Documentation: https://xcdat.readthedocs.io/en/stable/generated/xarray.Dataset.temporal.departures.html


In [ ]:
# Starter code here. When ready, click on the three dots below for the solution.
ds_global_anomaly = ds_global.temporal.departures()

In [ ]:
ds_global_anomaly = ds_global.temporal.departures(
    "TREFHT", freq="month", reference_period=("2000-01-01", "2009-12-31")
)

### Now let's plot the departures from the climatological average.


In [ ]:
plt.plot(dtime, ds_global_anomaly.TREFHT.values)
plt.xlabel("Year")
plt.ylabel("Global Mean Surface Temperature Anomaly [K]")

### Group averages

`xcdat` also allows you to calculate group averages (e.g., annual or seasonal mean from monthly data or monthly mean from daily data).


#### 💻 Your turn:

Calculate the annual mean from anomaly time series.

- API Documentation: https://xcdat.readthedocs.io/en/stable/generated/xarray.Dataset.temporal.group_averages.html


In [ ]:
# Starter code here. When ready, click on the three dots below for the solution.
ds_global_anomaly_annual = ds_global_anomaly.temporal.group_average()

In [ ]:
# compute annual mean from anomaly time series
ds_global_anomaly_annual = ds_global_anomaly.temporal.group_average(
    "TREFHT", freq="year"
)

### Now let's plot the results.


In [ ]:
# plot data
dtime_annual = dt2decimal(ds_global_anomaly_annual.time) + 0.5
plt.plot(
    dtime, ds_global_anomaly.TREFHT.values, label="Monthly departure", color="gray"
)
plt.plot(
    dtime_annual,
    ds_global_anomaly_annual.TREFHT.values,
    color="k",
    linestyle="",
    marker="_",
    label="Annual Mean",
)
plt.legend(frameon=False)
plt.xlabel("Year")
plt.ylabel("Global Mean Surface Temperature [K]")

## General Dataset Utilities

xCDAT includes various utilities for data manipulation, including
reorientation of the longitude axis, centering of time coordinates using time bounds, and adding and getting bounds.


### Reorient the longitude axis

Longitude can be represented from 0 to 360 E or as 180 W to 180 E. xcdat allows you to convert between these axes systems.


In [ ]:
ds.lon

#### 💻 Your turn:

Use `xc.swap_lon_axis` to swap the longitude axis from (0, 360) to (-180, 180) and view
the new longitude axis.

- Documentation: https://xcdat.readthedocs.io/en/stable/generated/xcdat.swap_lon_axis.html


In [ ]:
# Starter code here. When ready, click on the three dots below for the solution.
ds2 =

ds2.lon

In [ ]:
ds2 = xc.swap_lon_axis(ds, to=(-180, 180))

ds2.lon

### Add missing bounds

Bounds are critical to many `xcdat` operations. For example, they are used in determining the weights in spatial or temporal averages and in regridding operations. `add_missing_bounds()` will attempt to produce bounds if they do not exist in the original dataset.


In [ ]:
# We are dropping the existing bounds to demonstrate adding bounds.
ds4 = ds.drop_vars("time_bnds")

In [ ]:
try:
    ds4.bounds.get_bounds("T")
except KeyError as e:
    print(e)

#### 💻 Your turn:

Add the missing time bounds using `.bounds.add_missing_bounds()`.

- Documentation: https://xcdat.readthedocs.io/en/stable/generated/xarray.Dataset.bounds.add_missing_bounds.html
- Hint: Use the `axes` arg and pass a list containing a single string, `"T"` for time.


In [ ]:
# Starter code here. When ready, click on the three dots below for the solution.
ds5 = ds4.bounds.add_missing_bounds()

In [ ]:
ds5 = ds4.bounds.add_missing_bounds(axes=["T"])
ds5

## Interoperability with UXarray

UXarray provides Xarray-styled functionality for working with unstructured grids built around the UGRID conventions. Since UXarray's `ux.UxDataset` and `ux.UxDataArray` extend the `xr.Dataset` and `xr.DataArray` classes, _most_ xCDAT APIs are interoperable with UXarray objects.

- The exception is xCDAT's [spatial averager](https://xcdat.readthedocs.io/en/latest/generated/xarray.Dataset.spatial.average.html), which requires data on rectilinear grids. The data must first be remapped from unstructured to rectilinear grid using another tool like `nco` (`ncremap`).

## Next Steps

Congratulations on completing the xCDAT Practicum notebook for 2024 E3SM Tutorial
Workshop! If you haven't already, feel free to jump over to the `uxarray_practicum_notebook.ipynb`
to work with `uxarray`.
